In [ ]:
# lex_create_bot（ロールの作成）
import boto3
import time

iam = boto3.client('iam')
iam.create_service_linked_role(AWSServiceName='lex.amazonaws.com')
lex = boto3.client('lex-models', 'us-east-1')

In [ ]:
# lex_create_bot（スロットタイプの作成）
flavor_slot_type = lex.put_slot_type(
    name='FlavorSlotType',
    enumerationValues=[
        {'value': 'vanilla'},
        {'value': 'chocolate', 'synonyms': ['choc']},
        {'value': 'strawberry', 'synonyms': ['berry']}
    ],
    valueSelectionStrategy='TOP_RESOLUTION',
    createVersion=True)
print('slot type:', flavor_slot_type['name'])

container_slot_type = lex.put_slot_type(
    name='ContainerSlotType',
    enumerationValues=[
        {'value': 'corn'},
        {'value': 'cup'}
    ],
    valueSelectionStrategy='TOP_RESOLUTION',
    createVersion=True)
print('slot type:', container_slot_type['name'])


In [ ]:
# lex_create_bot（インテントの作成）
intent = lex.put_intent(
    name='OrderIntent',
    slots=[
        {
            'name': 'Flavor',
            'slotConstraint': 'Required',
            'slotType': 'FlavorSlotType',
            'slotTypeVersion': '1',
            'valueElicitationPrompt': {
                'messages': [{
                    'contentType': 'PlainText',
                    'content': 'Vanilla,chocolate or strawberry?'
                }],
                'maxAttempts': 3
            }
        },
        {
            'name': 'Container',
            'slotConstraint': 'Required',
            'slotType': 'ContainerSlotType',
            'slotTypeVersion': '1',
            'valueElicitationPrompt': {
                'messages': [{
                    'contentType': 'PlainText',
                    'content': 'Corn or cup?'
                }],
                'maxAttempts': 3
            }
        }
    ],
    sampleUtterances=[
        'I want {Flavor} ice cream in {Container}',
        '{Flavor} ice cream {Container}',
        'ice cream'
    ],
    conclusionStatement={
        'messages': [{
            'contentType': 'PlainText',
            'content': 'OK, {Flavor} ice cream in {Container}'
        }],
    },
    fulfillmentActivity={'type': 'ReturnIntent'},
    createVersion=True
)
print('intent:', intent['name'])


In [ ]:
# lex_create_bot（ボットの作成）

bot = lex.put_bot(
    name='MyBot', locale='en-US', childDirected=False,
    intents=[
        {
            'intentName': 'Order Intent',
            'intentVersion': '1'
        }
    ],
    abortStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Please try again.'
            }
        ]
    },
    voiceld='Joanna',
    createversion=True)
print('bot:', bot['name'])

start = time.time()
status = ''
while status not in ['READY', 'FAILED']:
    status = lex.get_bot(name='MyBot', versionOrAlias='1')['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time()-start, status))
if status == 'FAILED':
    print(lex.get_bot(
        name='MyBot', versionOrAlias='1')['failureReason'])

bot_alias = lex.put_bot_alias(
    name='MyBotAlias', botName='MyBot', botVersion='1'
)
print('bot alias:', bot_alias['name'])


In [ ]:
# lex_chat_text
import boto3
import uuid

lex_runtime = boto3.client('lex-runtime', 'us-east-1')
user = str(uuid.uuid1())
state = ''
while state != 'Fulfilled':
    result = lex_runtime.post_text(
        botName='MyBot',
        botAlias='MyBotAlias',
        userId=user,
        inputText=input('You:')
    )
    print('Bot:', result['message'])
    state = result['dialogState']
print()
print('Flavor:   ', result['slots']['Flavor'])
print('Container:', result['slots']['Container'])


In [ ]:
# lex_chat_voice
import boto3
import contextlib
import os
import uuid
lex_runtime = boto3.client('lex-runtime', 'us-east-1')
user = str(uuid.uuid1())
count = 0
state = ''

# ここに追記
while state != 'Fulfilled':
    wav = input('Wav:')
    with open(wav+'.wav', 'rb')as file:
        result = lex_runtime.post_content(
            botName='MyBot',
            botAlias='MyBotAlias',
            userId=user,
            accept='audio/mpeg',
            contentType='audio/116; rate=16000; channels=1',
            inputStream=file.read()
        )
    print('Bot:', result['message'])
    path = 'bot{}.mp3'.format(count)
    count += 1
    with contextlib.closing(result['audioStream'])as stream:
        with open(path, 'wb')as file:
            file.write(stream.read())
    if os.name == 'nt':
        os.startfile(path)
    state = result['dialogState']

print()
print('Flavor   :', result['slots']['Flavor'])
print('Container:', result['slots']['Container'])


In [ ]:
# lex_delete_bot
import boto3
import time

lex = boto3.client('lex-models', 'us-east-1')

for bot in lex.get_bots()['bots']:
    print('bot:', bot['name'])
    for bot_alias in lex.get_bot_aliases(
            botName=bot['name'])['BotAliases']:
        lex.delete_bot_alias(
            name=bot_alias['name'], botName=bot['name'])
    while True:
        try:
            lex.delete_bot(name=bot['name'])
            break
        except Exception:
            time.sleep(1)

for intent in lex.get_intents()['intents']:
    print('intent:', intent['name'])
    while True:
        try:
            lex.delete_intent(name=intent['name'])
            break
        except Exception:
            time.sleep(1)

for slot_type in lex.get_slot_types()['slotTypes']:
    print('slot type:', slot_type['name'])
    while True:
        try:
            lex.delete_slot_type(name=slot_type['name'])
            break
        except Exception:
            time.sleep(1)

iam = boto3.client('iam')
iam.delete_service_linked_role(RoleName='AWSServiceRoleForLexBots')


In [3]:
#音声ファイルを新しく作成する時はこれを利用
#（英語対応になっているので、日本語の時には音声IDを変更）
import boto3
import contextlib
import os
polly = boto3.client('polly', 'us-east-1')
text = input('Type a word: ')
result = polly.synthesize_speech(
    Text=text, OutputFormat='mp3', VoiceId='Justin',
    Engine='neural')
path = text + '.wav'
with contextlib.closing(result['AudioStream']) as stream:
    with open(path, 'wb') as file:
        file.write(stream.read())
if os.name == 'nt':
    os.startfile(path)

Type a word: test
